In [1]:
import pandas as pd
import spacy
import random
from collections import Counter
nlp = spacy.load("en_core_web_sm")


/home/users/jspiess/miniconda3/envs/codraw_pl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/codraw-icr-v2.tsv', delimiter='\t')
val_df = df[df['game_name'].str.startswith('val', na=False)]
val_df['game_id'] = val_df['game_name'].str.extract(r'_0*([1-9][0-9]*)')
val_df['turn'] = val_df['turn'].astype(str)
val_df.head()

/tmp/ipykernel_12792/3640256907.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['game_id'] = val_df['game_name'].str.extract(r'_0*([1-9][0-9]*)')
/tmp/ipykernel_12792/3640256907.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['turn'] = val_df['turn'].astype(str)


,Unnamed: 0,teller_before,drawer,teller_after,is_CR_annotator_1,do_annotators_agree,is_CR_annotator_2,mood,is_source_utterance_last_turn,next_turn_contains_response,...,size,direction,relation_to_other_cliparts,disambig_object,disambig_person,game_name,turn,annotation_round,freq,game_id
7,8,next on the left of the dog is a sandbox,does the sandbox go down to the bottom or wher...,the tree is in back of the sandbox . a cat in ...,1,1,1,"polar question,wh-question",1,0,...,0,0,1,0,0,val_09368,3,all types,1.0,9368
33,61,make tree bigger,put it medium size now . is it ok ?,make picnic table bigger and in the center,0,0,1,"declarative,polar question",1,0,...,1,0,0,0,0,val_07918,11,all types,1.0,7918
34,65,yes,"is the left side of the girl 's leg , on your ...",edge,0,0,1,polar question,0,1,...,0,0,1,0,0,val_06878,13,all types,1.0,6878
49,91,girl sitting on the left white blue pointy hat...,girl is directly under helicopter ?,"heli facing right , guy across the girl sittin...",1,1,1,polar question,1,0,...,0,0,1,0,0,val_09678,1,all types,1.0,9678
71,141,yes,med ?,there is a shocked boy sitting at the end of t...,0,0,1,other,0,0,...,1,0,0,0,0,val_08028,3,all types,1.0,8028


In [3]:
with open("./outputs/val_games.txt") as file:
    games = [line.strip() for line in file.readlines()]
    
with open("./outputs/val_turns.txt") as file:
    turns = [line.strip() for line in file.readlines()]

In [4]:
output_df = pd.DataFrame({'game_id': games, 'turn': turns})
output_df

,game_id,turn
0,2468,3
1,2468,4
2,2468,5
3,5868,1
4,9368,1
...,...,...
909,7698,2
910,6928,2
911,6928,3
912,6928,5


In [5]:
# Merge DataFrames based on 'game_id' and 'turn'
merged_df = pd.merge(output_df, val_df, on=['game_id', 'turn'])

columns_to_keep = ['game_id', 'turn', 'teller_before', "drawer", "position", "size", "direction", "relation_to_other_cliparts", "disambig_object", "disambig_person"]
final_df = merged_df[columns_to_keep]
final_df

,game_id,turn,teller_before,drawer,position,size,direction,relation_to_other_cliparts,disambig_object,disambig_person
0,2468,3,right side of picture is rocket flames just ba...,you mean rocket is left or behind the tree ?,1,0,0,1,0,0
1,2468,4,about inch half up from grass but can see whol...,but where is it ? left edge or behind the tree ?,1,0,0,1,0,0
2,2468,5,rocket is far right of picture in sky,so tree is right too you said ?,1,0,0,0,0,0
3,5868,1,big pine tree on the left with a boy sitting i...,what hat,0,0,0,0,1,0
4,9368,1,there is a park with a dog .,where is the dog located ?,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
909,7698,2,on the right is a medium boy facing the right ...,on right of girl ?,0,0,0,1,1,0
910,6928,2,there is a small helicopter to the right of th...,facing where ? size ?,0,1,1,0,0,0
911,6928,3,"she is facing left , she is i guess big , her ...","size , direction facing for helicopter and duck ?",0,1,1,0,0,0
912,6928,5,there is a big boy with a hat in front of the ...,is he large ?,0,1,0,0,0,0


In [6]:
def extract_first_noun(text):
    if not isinstance(text, str):
        return ''

    doc = nlp(text)
    for token in doc:
        if token.pos_ == 'NOUN':
            return token.text
    return ''

def create_new_column(row, distribution, options):
    first_noun = extract_first_noun(row['teller_before'])
    chosen_option = random.choices(options, weights=distribution, k=1)[0]
    return f"is the {first_noun} {chosen_option} ?"

options = ["position", "size", "direction", "relation", "what", "what"]
column_names = ["position", "size", "direction", "relation_to_other_cliparts", "disambig_object", "disambig_person"]

weights_for_what = final_df[column_names[-2]].sum() + final_df[column_names[-1]].sum()
distribution = final_df[column_names[:-2]].sum().tolist() + [weights_for_what, weights_for_what]

final_df['baseline1'] = final_df.apply(lambda row: create_new_column(row, distribution, options), axis=1)

final_df

/tmp/ipykernel_12792/1135176404.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['baseline1'] = final_df.apply(lambda row: create_new_column(row, distribution, options), axis=1)


,game_id,turn,teller_before,drawer,position,size,direction,relation_to_other_cliparts,disambig_object,disambig_person,baseline1
0,2468,3,right side of picture is rocket flames just ba...,you mean rocket is left or behind the tree ?,1,0,0,1,0,0,is the side what ?
1,2468,4,about inch half up from grass but can see whol...,but where is it ? left edge or behind the tree ?,1,0,0,1,0,0,is the inch direction ?
2,2468,5,rocket is far right of picture in sky,so tree is right too you said ?,1,0,0,0,0,0,is the rocket size ?
3,5868,1,big pine tree on the left with a boy sitting i...,what hat,0,0,0,0,1,0,is the tree position ?
4,9368,1,there is a park with a dog .,where is the dog located ?,1,0,0,0,0,0,is the park position ?
...,...,...,...,...,...,...,...,...,...,...,...
909,7698,2,on the right is a medium boy facing the right ...,on right of girl ?,0,0,0,1,1,0,is the right position ?
910,6928,2,there is a small helicopter to the right of th...,facing where ? size ?,0,1,1,0,0,0,is the helicopter direction ?
911,6928,3,"she is facing left , she is i guess big , her ...","size , direction facing for helicopter and duck ?",0,1,1,0,0,0,is the mouth size ?
912,6928,5,there is a big boy with a hat in front of the ...,is he large ?,0,1,0,0,0,0,is the boy what ?


In [7]:
all_tokens = [token for row in final_df['drawer'] for token in row.split()]
token_distribution = Counter(all_tokens)

length_distribution = final_df['drawer'].apply(lambda x: len(x.split())).tolist()

def create_random_string(token_distribution, length_distribution):
    length = random.choice(length_distribution)
    return ' '.join(random.choices(list(token_distribution.keys()), 
                                   weights=token_distribution.values(), k=length))

final_df['baseline2'] = final_df.apply(lambda row: create_random_string(token_distribution, length_distribution), axis=1)

final_df


/tmp/ipykernel_12792/2762385980.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['baseline2'] = final_df.apply(lambda row: create_random_string(token_distribution, length_distribution), axis=1)


,game_id,turn,teller_before,drawer,position,size,direction,relation_to_other_cliparts,disambig_object,disambig_person,baseline1,baseline2
0,2468,3,right side of picture is rocket flames just ba...,you mean rocket is left or behind the tree ?,1,0,0,1,0,0,is the side what ?,girl takes she of beach ? above side or his sm...
1,2468,4,about inch half up from grass but can see whol...,but where is it ? left edge or behind the tree ?,1,0,0,1,0,0,is the inch direction ?,direction table what cloud ?
2,2468,5,rocket is far right of picture in sky,so tree is right too you said ?,1,0,0,0,0,0,is the rocket size ?,"standing next off that , snake sad is ? paws"
3,5868,1,big pine tree on the left with a boy sitting i...,what hat,0,0,0,0,1,0,is the tree position ?,duck ? how center it is touching side girl ? s...
4,9368,1,there is a park with a dog .,where is the dog located ?,1,0,0,0,0,0,is the park position ?,cloud or is of no she hands .
...,...,...,...,...,...,...,...,...,...,...,...,...
909,7698,2,on the right is a medium boy facing the right ...,on right of girl ?,0,0,0,1,1,0,is the right position ?,"hole in , ? cloud ? ok is of to"
910,6928,2,there is a small helicopter to the right of th...,facing where ? size ?,0,1,1,0,0,0,is the helicopter direction ?,they indian tree that is
911,6928,3,"she is facing left , she is i guess big , her ...","size , direction facing for helicopter and duck ?",0,1,1,0,0,0,is the mouth size ?,the face she is ?
912,6928,5,there is a big boy with a hat in front of the ...,is he large ?,0,1,0,0,0,0,is the boy what ?,"table and be tree ball , are and , girl girl m..."


In [8]:
def create_new_column(row, distribution, options):
    drawer_tokens = row['drawer'].split()  
    drawer_vocab = Counter(drawer_tokens)  

    chosen_tokens = random.choices(list(drawer_vocab.keys()), weights=list(drawer_vocab.values()), k=2)

    first_noun = extract_first_noun(row['teller_before'])
    chosen_option = random.choices(options, weights=distribution, k=1)[0]

    new_string = f"is the {first_noun} {chosen_option} "
    new_string += ' '.join(chosen_tokens)  
    new_string += ' ?'

    return new_string

final_df['baseline3'] = final_df.apply(lambda row: create_new_column(row, distribution, options), axis=1)
final_df

/tmp/ipykernel_12792/3475641249.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['baseline3'] = final_df.apply(lambda row: create_new_column(row, distribution, options), axis=1)


,game_id,turn,teller_before,drawer,position,size,direction,relation_to_other_cliparts,disambig_object,disambig_person,baseline1,baseline2,baseline3
0,2468,3,right side of picture is rocket flames just ba...,you mean rocket is left or behind the tree ?,1,0,0,1,0,0,is the side what ?,girl takes she of beach ? above side or his sm...,is the side relation left tree ?
1,2468,4,about inch half up from grass but can see whol...,but where is it ? left edge or behind the tree ?,1,0,0,1,0,0,is the inch direction ?,direction table what cloud ?,is the inch relation but it ?
2,2468,5,rocket is far right of picture in sky,so tree is right too you said ?,1,0,0,0,0,0,is the rocket size ?,"standing next off that , snake sad is ? paws",is the rocket relation you tree ?
3,5868,1,big pine tree on the left with a boy sitting i...,what hat,0,0,0,0,1,0,is the tree position ?,duck ? how center it is touching side girl ? s...,is the tree direction hat hat ?
4,9368,1,there is a park with a dog .,where is the dog located ?,1,0,0,0,0,0,is the park position ?,cloud or is of no she hands .,is the park what the the ?
...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,7698,2,on the right is a medium boy facing the right ...,on right of girl ?,0,0,0,1,1,0,is the right position ?,"hole in , ? cloud ? ok is of to",is the right what right girl ?
910,6928,2,there is a small helicopter to the right of th...,facing where ? size ?,0,1,1,0,0,0,is the helicopter direction ?,they indian tree that is,is the helicopter what ? ? ?
911,6928,3,"she is facing left , she is i guess big , her ...","size , direction facing for helicopter and duck ?",0,1,1,0,0,0,is the mouth size ?,the face she is ?,is the mouth relation and direction ?
912,6928,5,there is a big boy with a hat in front of the ...,is he large ?,0,1,0,0,0,0,is the boy what ?,"table and be tree ball , are and , girl girl m...",is the boy relation he he ?


In [ ]:
file_name = "Baseline1.txt"

with open(file_name, 'w') as file:
    for string in final_df["baseline1"].tolist():
        file.write(string + '\n')

print(f"Baseline1 written to {file_name}")


In [ ]:
file_name = "Baseline2.txt"

with open(file_name, 'w') as file:
    for string in final_df["baseline2"].tolist():
        file.write(string + '\n')

print(f"Baseline2 written to {file_name}")


In [ ]:
file_name = "Baseline3.txt"

with open(file_name, 'w') as file:
    for string in final_df["baseline3"].tolist():
        file.write(string + '\n')

print(f"Baseline3 written to {file_name}")